In [13]:
import argparse
import os
import pandas as pd
import snowflake.connector
import json
import boto3
import xxhash
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.metrics import roc_auc_score
import numpy as np

from typing import Dict

from tqdm import tqdm


sf_account_id = "lnb99345.us-east-1"
sf_secret_id = "snowflake_credentials"
warehouse = "XSMALL"
database = "PRODUCTION"
schema = "SIGNALS"


In [14]:
def get_credentials(secret_id: str, region_name: str) -> str:
    session = boto3.session.Session(profile_name="ml-staging-admin")
    client = session.client('secretsmanager', region_name=region_name)
    response = client.get_secret_value(SecretId=secret_id)
    secrets_value = json.loads(response['SecretString'])    
    
    return secrets_value

In [15]:
session = boto3.session.Session(profile_name="ml-staging-admin")
region = session.region_name
region

'us-east-1'

In [16]:
def connect(secret_id: str, account: str, warehouse: str, database: str, schema: str, region: str) -> snowflake.connector.SnowflakeConnection:
    
    secret_value = get_credentials(secret_id, region)
    sf_user = secret_value['username']
    sf_password = secret_value['password']
    sf_account = account
    sf_warehouse = warehouse
    sf_database = database
    sf_schema = schema
    sf_protocol = "https"
    
    print(f"sf_user={sf_user}, sf_password=****, sf_account={sf_account}, sf_warehouse={sf_warehouse}, "
          f"sf_database={sf_database}, sf_schema={sf_schema}, sf_protocol={sf_protocol}")    
    
    # Read to connect to snowflake
    ctx = snowflake.connector.connect(user=sf_user,
                                      password=sf_password,
                                      account=sf_account,
                                      warehouse=sf_warehouse,
                                      database=sf_database,
                                      schema=sf_schema,
                                      protocol=sf_protocol)
    
    return ctx

In [17]:
ctx = connect(sf_secret_id, sf_account_id, warehouse, database, schema, "us-east-1")
ctx

sf_user=MGAIDUK, sf_password=****, sf_account=lnb99345.us-east-1, sf_warehouse=XSMALL, sf_database=PRODUCTION, sf_schema=SIGNALS, sf_protocol=https


In [18]:
def collect_dataset(ctx: snowflake.connector.SnowflakeConnection, input: str) -> pd.DataFrame:
    # Collect dataset
    sql = f"""
    select * from {input}
    """
    df = pd.read_sql(sql, ctx)
    return df

df = collect_dataset(ctx, input="TRAIN_DATASET")
df

/tmp/ipykernel_3457283/1686504616.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, ctx)


,USERID,MEDIAID,MEDIATAKENBYID,REQUESTID,NVIEWS,TIMESPENT,MIN_TIMESTAMP,NREACTIONS
0,f34d7147-9d39-4145-bc7e-3a3c05029035,01HEKK737WXFC9TWQ036XDS0QH,3f8d2288-5c75-4661-8026-66bfa11c49e2,95151a9d-b5f1-4ab7-9b44-56098337d4f4,1,0.53772,2024-02-20 02:55:30.706,0
1,46b5b420-3317-4a14-9c90-9dc4206cf960,01H2K82V8XCKGD882FPQ3TQ9WF,0389e482-7b73-4b85-b07c-97b8fdfa27ed,d7c1a5a0-d19d-4372-8982-4d5172e3d242,1,24.64895,2024-02-20 02:55:22.873,0
2,f34d7147-9d39-4145-bc7e-3a3c05029035,01HDZ1090BMQGT4AN0NNC54MWA,2f3448e0-2815-40dc-bbd5-e75f751c2c17,22e44e28-7992-48a6-a5c5-bec468d24faa,1,1.21383,2024-02-20 02:55:54.587,0
3,ebf6fff3-93e6-49bd-8eae-5d800a5ffc1e,01H7C16YSARTCVC42H97Y2PNGT,0584f3b8-a5bd-437f-aeb1-04a5202f2edc,ec849d64-24a8-46f5-8fb9-82e7abd3b11d,1,2.63363,2024-02-20 02:54:52.774,0
4,9aa96427-66df-4820-aa6d-7cbfd76f5977,01HK24THNFWPGERGMWAVVE8GNS,1dbec9e0-338b-40aa-acef-823144f0fc47,ae56e19a-6137-48c8-907f-d9dec7b6e282,1,2.09774,2024-02-20 02:55:00.122,0
...,...,...,...,...,...,...,...,...
3569046,5d3fac96-1a54-4bb0-8af9-2c9813d16902,01HGQ21YNDG61PK8W5G5ZXRHMF,7895ced7-7d77-4b35-abd0-bc5a87568755,a2af331c-6680-433d-bda8-b95122f2d3bc,0,0.00000,2024-03-02 19:53:41.087,0
3569047,0929b643-7862-493f-86ef-27f1fc0a0794,01H5H8C3BDN8Q5E8RS43XRXYHE,a216580a-5fb7-4d79-8ce2-3f5e40963005,3058b9af-9c05-412f-adec-efbfdff2467d,0,0.00000,2024-03-02 19:53:44.355,0
3569048,ad46903d-975f-4fad-b463-886fc90deb2e,01H5GH6JZMF60THS9XN882MF7H,1ed23824-14c1-4193-8e92-2bc2a5351103,99d920bb-91e3-4ffb-b6f9-588bf4c43212,1,1.94881,2024-03-02 19:54:56.918,1
3569049,3beac821-902f-4504-b308-143e6d5bd84c,01HKWD0AREJVRV90J3870NZQ98,32c316ea-022b-4fe3-9336-987cd27d8a0b,24fddea2-eb0c-43f4-ba53-648a817cae48,0,0.00000,2024-03-02 19:55:31.534,0


In [19]:
df.shape

(3569051, 8)

In [20]:
df = df.drop("REQUESTID", axis=1)
df

,USERID,MEDIAID,MEDIATAKENBYID,NVIEWS,TIMESPENT,MIN_TIMESTAMP,NREACTIONS
0,f34d7147-9d39-4145-bc7e-3a3c05029035,01HEKK737WXFC9TWQ036XDS0QH,3f8d2288-5c75-4661-8026-66bfa11c49e2,1,0.53772,2024-02-20 02:55:30.706,0
1,46b5b420-3317-4a14-9c90-9dc4206cf960,01H2K82V8XCKGD882FPQ3TQ9WF,0389e482-7b73-4b85-b07c-97b8fdfa27ed,1,24.64895,2024-02-20 02:55:22.873,0
2,f34d7147-9d39-4145-bc7e-3a3c05029035,01HDZ1090BMQGT4AN0NNC54MWA,2f3448e0-2815-40dc-bbd5-e75f751c2c17,1,1.21383,2024-02-20 02:55:54.587,0
3,ebf6fff3-93e6-49bd-8eae-5d800a5ffc1e,01H7C16YSARTCVC42H97Y2PNGT,0584f3b8-a5bd-437f-aeb1-04a5202f2edc,1,2.63363,2024-02-20 02:54:52.774,0
4,9aa96427-66df-4820-aa6d-7cbfd76f5977,01HK24THNFWPGERGMWAVVE8GNS,1dbec9e0-338b-40aa-acef-823144f0fc47,1,2.09774,2024-02-20 02:55:00.122,0
...,...,...,...,...,...,...,...
3569046,5d3fac96-1a54-4bb0-8af9-2c9813d16902,01HGQ21YNDG61PK8W5G5ZXRHMF,7895ced7-7d77-4b35-abd0-bc5a87568755,0,0.00000,2024-03-02 19:53:41.087,0
3569047,0929b643-7862-493f-86ef-27f1fc0a0794,01H5H8C3BDN8Q5E8RS43XRXYHE,a216580a-5fb7-4d79-8ce2-3f5e40963005,0,0.00000,2024-03-02 19:53:44.355,0
3569048,ad46903d-975f-4fad-b463-886fc90deb2e,01H5GH6JZMF60THS9XN882MF7H,1ed23824-14c1-4193-8e92-2bc2a5351103,1,1.94881,2024-03-02 19:54:56.918,1
3569049,3beac821-902f-4504-b308-143e6d5bd84c,01HKWD0AREJVRV90J3870NZQ98,32c316ea-022b-4fe3-9336-987cd27d8a0b,0,0.00000,2024-03-02 19:55:31.534,0


In [10]:
def my_hash(s: str) -> int:
    return xxhash.xxh32(s).intdigest()

df[['USERID', 'MEDIAID', 'MEDIATAKENBYID']] = df[['USERID', 'MEDIAID', 'MEDIATAKENBYID']].applymap(my_hash)
df

/tmp/ipykernel_3457283/573130622.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[['USERID', 'MEDIAID', 'MEDIATAKENBYID']] = df[['USERID', 'MEDIAID', 'MEDIATAKENBYID']].applymap(my_hash)


,USERID,MEDIAID,MEDIATAKENBYID,NVIEWS,TIMESPENT,MIN_TIMESTAMP,NREACTIONS
0,2801485989,437423269,198149162,1,0.53772,2024-02-20 02:55:30.706,0
1,3674906565,306905446,3251702794,1,24.64895,2024-02-20 02:55:22.873,0
2,2801485989,2680684633,992631802,1,1.21383,2024-02-20 02:55:54.587,0
3,2785161045,2981110049,719582003,1,2.63363,2024-02-20 02:54:52.774,0
4,99142180,3773461872,1636118249,1,2.09774,2024-02-20 02:55:00.122,0
...,...,...,...,...,...,...,...
3569046,2218615370,3773753290,1501251771,0,0.00000,2024-03-02 19:53:41.087,0
3569047,3789865737,1644065753,1313345833,0,0.00000,2024-03-02 19:53:44.355,0
3569048,3983981813,2232497766,3600978512,1,1.94881,2024-03-02 19:54:56.918,1
3569049,4242759565,3803287101,3934434459,0,0.00000,2024-03-02 19:55:31.534,0


In [11]:
from datetime import datetime, timedelta
yesterday = datetime.now() - timedelta(days=1)

val_df = df[df["MIN_TIMESTAMP"] > yesterday.strftime('%Y-%m-%d')]
train_df = df[df["MIN_TIMESTAMP"] <= yesterday.strftime('%Y-%m-%d')]
train_df.shape, val_df.shape

((3195461, 7), (373590, 7))

In [12]:
train_df

,USERID,MEDIAID,MEDIATAKENBYID,NVIEWS,TIMESPENT,MIN_TIMESTAMP,NREACTIONS
0,2801485989,437423269,198149162,1,0.53772,2024-02-20 02:55:30.706,0
1,3674906565,306905446,3251702794,1,24.64895,2024-02-20 02:55:22.873,0
2,2801485989,2680684633,992631802,1,1.21383,2024-02-20 02:55:54.587,0
3,2785161045,2981110049,719582003,1,2.63363,2024-02-20 02:54:52.774,0
4,99142180,3773461872,1636118249,1,2.09774,2024-02-20 02:55:00.122,0
...,...,...,...,...,...,...,...
3569046,2218615370,3773753290,1501251771,0,0.00000,2024-03-02 19:53:41.087,0
3569047,3789865737,1644065753,1313345833,0,0.00000,2024-03-02 19:53:44.355,0
3569048,3983981813,2232497766,3600978512,1,1.94881,2024-03-02 19:54:56.918,1
3569049,4242759565,3803287101,3934434459,0,0.00000,2024-03-02 19:55:31.534,0


In [15]:
df.drop(["NVIEWS", "MIN_TIMESTAMP"], axis=1)

,USERID,MEDIAID,MEDIATAKENBYID,TIMESPENT,NREACTIONS
0,981375360,550619802,693557974,0.00000,0
1,981375360,2859121093,311857850,2.48329,0
2,981375360,1049864589,1885778205,0.84982,0
3,312633046,1496773919,474122613,2.26662,0
4,2438299337,180604616,853446312,0.00000,0
...,...,...,...,...,...
3279486,3177464199,3129286366,2181173437,0.86511,0
3279487,159648535,2369744197,1032770509,1.40263,0
3279488,241687729,282749124,3606379008,1.09249,0
3279489,3924562217,540872740,1265141866,0.73330,0


In [16]:
print(torch.cuda.is_available())
device = "cuda"

True


In [17]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, device):
        self.dataframe = dataframe
        self.features = torch.tensor(dataframe[['USERID', 'MEDIAID', 'MEDIATAKENBYID']].values, dtype=torch.int32)
        self.labels = torch.tensor(dataframe[['TIMESPENT', 'NREACTIONS']].values, dtype=torch.float32)

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        features = self.features[idx]
        labels = self.labels[idx]
        has_reactions = (labels[1] > 0).float()
        return {
            'features': {'USERID': features[0], 'MEDIAID': features[1], 'MEDIATAKENBYID': features[2]},
            'labels': {'TIMESPENT': labels[0], 'HAS_REACTIONS': has_reactions}
        }

In [26]:
train_dt = CustomDataset(train_df, device=device)
val_dt = CustomDataset(val_df, device=device)
batch_size = 4096  # Adjust based on your needs
shuffle = True  # Shuffle the data
num_workers = 2
train_dataloader = DataLoader(train_dt, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True)
val_dataloader = DataLoader(val_dt, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

In [19]:
for batch in train_dataloader:
    features = batch['features']
    labels = batch['labels']
    print(f"Features: {features}, Labels: {labels}")
    break

Features: {'USERID': tensor([  891617750, -1465996505,   677667058,  ..., -2131328559,
        -2110256454, -1135862032], dtype=torch.int32), 'MEDIAID': tensor([ 1309490148,   573819180, -1372872497,  ...,  1322138521,
        -1735292343,   418718718], dtype=torch.int32), 'MEDIATAKENBYID': tensor([ 1816451511,  -536419951,   781585977,  ...,  1698903367,
        -1881626219, -1887074661], dtype=torch.int32)}, Labels: {'TIMESPENT': tensor([0.0000, 1.0007, 1.4507,  ..., 1.6204, 2.9163, 0.3838]), 'HAS_REACTIONS': tensor([0., 0., 0.,  ..., 0., 1., 0.])}


In [20]:
import math

class CollaborativeFilteringModel(nn.Module):
    def __init__(self, num_users, num_posts, embedding_dim):
        super(CollaborativeFilteringModel, self).__init__()
        self.num_users = num_users
        self.num_posts = num_posts
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.post_embedding = nn.Embedding(num_posts, embedding_dim)
        
    def forward(self, user_ids, post_ids):
        user_ids %= self.num_users
        post_ids %= self.num_posts
        user_embedded = self.user_embedding(user_ids)
        post_embedded = self.post_embedding(post_ids)
        interaction = (user_embedded * post_embedded).sum(dim=1)
        prediction = torch.sigmoid(interaction)
        return prediction

In [21]:
class CollaborativeFilteringModel2(nn.Module):
    def __init__(self, num_users, num_posts, num_creators, embedding_dim):
        super(CollaborativeFilteringModel2, self).__init__()
        self.num_users = num_users
        self.num_posts = num_posts
        self.num_creators = num_creators
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.post_embedding = nn.Embedding(num_posts, embedding_dim)
        self.creator_embedding = nn.Embedding(num_creators, embedding_dim)
        self.dense = nn.Linear(in_features= 2, out_features=1)
        
    def forward(self, user_ids, post_ids, creator_ids):
        user_ids %= self.num_users
        post_ids %= self.num_posts
        creator_ids %= self.num_creators
        user_embedding = self.user_embedding(user_ids)
        post_embedding = self.post_embedding(post_ids)
        creator_embedding = self.creator_embedding(creator_ids)
        interaction1 = (user_embedding * post_embedding).sum(dim=1)
        interaction2 = (user_embedding * creator_embedding).sum(dim=1)
        prediction = torch.sigmoid(self.dense(interaction1, interaction2))
        return prediction

In [22]:
def dict_to_device(dic: Dict, device: str) -> Dict:
    return {k: v.to(device) for k, v in dic.items()}

In [23]:
import wandb
wandb.login(key="5558b41450b126a5fc0e47c271d4252b06e32fe6")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adensur. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/mak/.netrc


True

In [24]:
print(len(df["USERID"].unique()), len(df["MEDIAID"].unique()), len(df["MEDIATAKENBYID"].unique()))

373271 1930 1455


In [27]:
# train lööp
print(len(df["USERID"].unique()), len(df["MEDIAID"].unique()))
num_users = 50000
num_posts = 1800
num_creators = 700
embedding_dim = 4
model = CollaborativeFilteringModel(num_users=num_users, num_posts=num_posts, embedding_dim=embedding_dim).to(device)

criterion = nn.BCELoss()
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)

num_epochs = 500
wandb.init(
      # Set the project where this run will be logged
      project="lapse-cf-pytorch-3", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"creator_embeds_small_dicts", 
      # Track hyperparameters and run metadata
      config={
        "learning_rate": lr,
        "architecture": "CF_user_post",
        "epochs": num_epochs,
        "train_start": train_df["MIN_TIMESTAMP"].min(),
        "train_end": train_df["MIN_TIMESTAMP"].max(),
        "train_count": train_df.shape[0],
        "val_start": val_df["MIN_TIMESTAMP"].min(),
        "val_end": val_df["MIN_TIMESTAMP"].max(),
        "val_count": val_df.shape[0],
        "num_users": num_users,
        "num_posts": num_posts,
        "embedding_dim": embedding_dim,
        "batch_size": batch_size,
        "init": "uniform/4",
      })


for epoch in range(num_epochs):
    for batch in tqdm(train_dataloader):
        features = batch['features']
        labels = batch['labels']
        features = dict_to_device(features, device)
        labels = dict_to_device(labels, device)
        optimizer.zero_grad()
        outputs = model(features["USERID"], features["MEDIAID"])
        targets = labels["HAS_REACTIONS"]
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1}, train loss: {loss.item()}')
    wandb.log({"train_loss": loss.item(), "epoch": epoch+1})

    # Validation phase
    model.eval() # Set the model to evaluation mode
    val_loss = 0.0
    all_targets = []
    all_outputs = []
    with torch.no_grad():
         for batch in val_dataloader:
            features = batch['features']
            labels = batch['labels']
            features = dict_to_device(features, device)
            labels = dict_to_device(labels, device)
            outputs = model(features["USERID"], features["MEDIAID"])
            targets = labels["HAS_REACTIONS"]
            loss = criterion(outputs, targets)
            val_loss += loss.item()
            all_targets.extend(targets.cpu().numpy())  # Collect all targets
            all_outputs.extend(outputs.cpu().numpy())  # Collect all outputs
    auc_roc_score = roc_auc_score(all_targets, all_outputs)
    
    # Print average validation loss per epoch
    print(f'Epoch {epoch+1}, Validation Loss: {val_loss / len(val_dataloader)}, AUC-ROC: {auc_roc_score}')
    wandb.log({"val_loss": loss.item(), "epoch": epoch+1, "roc-auc": auc_roc_score})

373271 1930


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


100%|██████████| 720/720 [00:50<00:00, 14.28it/s]

Epoch 1, train loss: 0.8678579926490784


Epoch 1, Validation Loss: 0.9008141292466058, AUC-ROC: 0.49644009736071293


...

Epoch 499, train loss: 0.11614155024290085


Epoch 499, Validation Loss: 0.39808689149809473, AUC-ROC: 0.6801701541865082


100%|██████████| 720/720 [00:50<00:00, 14.16it/s]

Epoch 500, train loss: 0.11498032510280609


Epoch 500, Validation Loss: 0.39810824063089156, AUC-ROC: 0.6800909325903546
